# Desafio +A Educação - Engenheiro de Inteligência Artificial

Este notebook contempla as etapas e soluções requeridas no desafio pontuadas no README.md na pasta raiz e estruturadas de acordo

*Autora*: Leticia Campos Valente

## Etapa 1: Indexação dos dados

**Enunciado**:\
"Defina a ferramenta de indexação que considere adequada para o projeto. Recomenda-se escolher uma ferramenta que possa suportar a geração dinâmica de conteúdos adaptativos no prompt de IA generativa. Indexe os diferentes tipos de dados para permitir uma busca eficiente e relevante."

**Conjunto de dados**: \
Textos: Uma coleção de textos extraídos de módulos de aprendizagem.
PDFs: Livros e manuais introdutórios sobre o conteúdo abordado.
Vídeos: Pequenos vídeos de dicas de professor, que explicam o conteúdo abordado. Os vídeos estão em formatos .mp4


Dadas as condições, opta-se pela ferramenta ElasticSearch pela eficiencia no armazenamento de documentação e integração em Langchain

 <u> Notas </u>

1. Existem nesse código mais de uma estratégia para realizar, por exemplo, extração das keywords. Isso se dá porque o mesmo método que performa bem para o texto de exemplo não performou tão bem para os textos em pdf, etc. Além disso, inclui-se mais abordagens propositalmente para demonstrar os conhecimentos em cada estratégia
2.

### 1.1 Pré-processamento de Texto

É preciso primeiramente extrair as palavras chaves do texto para indexação conforme conteúdo e extração de frases relevantes. 

**Estratégia de extração de palavras-chave**: Term Frequency-Inverse Document Frequency (TF-IDF) \
**Estratégia de extração de frases mais relevantes**: Sentence ranking via spacy 

Escolheu-se utilizar abordagens de NLP simples ao invés do uso de um modelo pré-treinado BERT, por exemplo, por mera questão de simplicidade e eficiência para este documento de exemplo, mas em textos mais complexos, a autora consideraria a segunda opção.

As funções se encontram no módulo source/text_utils.py

In [1]:
import nltk
from text_utils import latin_text_preprocessing, tfidf_keyword_extraction, text_most_relevant_prhases_extraction, extract_programming_language

In [2]:
%%capture
!python -m spacy download pt_core_news_sm

In [3]:
%%capture
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\letic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\letic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### Execução

In [4]:
text_path = '../resources/Apresentação.txt'
with open(text_path, 'r') as text_file:
    text_data = text_file.read()

In [5]:
text_doc = {
    'content':str(latin_text_preprocessing(text_data)),
    'keywords':tfidf_keyword_extraction(text_data),
    'relevant_phrases':text_most_relevant_prhases_extraction(text_data),
    'programming_languages':extract_programming_language(text_data),
    'text_path':text_path,
    'format':'text'
}

In [6]:
text_doc

{'content': 'Sistemas desenvolvidos para a plataforma web utilizam a linguagem HTML para a exibição de conteúdo, tanto para páginas estáticas como dinâmicas. Com HTML5 é possível criar páginas web com diversos recursos para a apresentação de dados por meio de novas marcações que permitem o uso de semântica e acessibilidade, facilitando a pesquisa por motores de busca automática e dispositivos próprios para deficientes visuais e auditivos.\n\nNesta Unidade de Aprendizagem, você vai estudar a estrutura de páginas web, a formatação de texto em documentos hipertexto e a apresentação de links, listas e tabelas em HTML5.\n\nBons estudos.\n\nAo final desta Unidade de Aprendizagem, você deve apresentar os seguintes aprendizados:\nDefinir a estrutura de uma página web com HTML5.\nAplicar formatação de texto em uma página web com HTML5.\nDesenvolver listas e tabelas em uma página web com HTML5.',
 'keywords': ['página',
  'web',
  'html5',
  'aprendizagem',
  'apresentação',
  'desenvolver',
  '

### 1.2 Pré-processamento de PDF

**Estratégia de extração de palavras-chave**: YAKE (Yet another keyword extractor) \
**Estratégia de extração de autor do texto**: NER ( Named Entity Recognition)


As funções se encontram no módulo source/text_utils.py

In [7]:
from text_utils import yake_keyword_extraction, ner_author_extractor
from PyPDF2 import PdfReader

##### Execução

In [8]:
pdf_path = '../resources/Capítulo do Livro.pdf'
with open(pdf_path, 'rb') as pdf_file:
    # Criação do objeto PdfReader
    pdf_reader = PdfReader(pdf_file)

    extracted_text = ""

    # Extração do texto do pdf por páginas
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        extracted_text += page.extract_text()

unknown widths : 
[0, IndirectObject(176, 0, 2167199911264)]


In [9]:
pdf_doc = {
    'content':extracted_text,
    'keywords':yake_keyword_extraction(extracted_text),
    'programming_languages':extract_programming_language(extracted_text),
    'pdf_path':pdf_path,
    'pdf_author':ner_author_extractor(extracted_text),
    'format':'pdf'
}

In [10]:
pdf_doc

{'content': 'DESENVOLVIMENTO \nDE SISTEMAS PHP \nMaurício de Oliveira Saraiva Criação de páginas web \ncom HTML5\nObjetivos de aprendizagem\nAo final deste texto, você deve apresentar os seguintes aprendizados:\n \x84Definir a estrutura de uma página web com HTML5.\n \x84Aplicar a formatação de texto em uma página web com HTML5.\n \x84Desenvolver listas e tabelas em uma página web com HTML5.\nIntrodução\nSistemas desenvolvidos para a plataforma web, tanto para páginas es -\ntáticas como dinâmicas, utilizam a linguagem HTML para a exibição \ndo conteúdo. Com HTML5, é possível criar páginas web com diversos \nrecursos para a apresentação de dados, por meio de novas marcações \nque permitem o uso de semântica e acessibilidade. Isso facilita a pesquisa \npor motores de busca automática, dispositivos próprios para deficientes \nvisuais e auditivos, entre outros.\nNeste capítulo, você vai estudar a estrutura de páginas web, a for -\nmatação de texto em documentos hipertexto e apresentação de

### 1.3 Pré-processamento de vídeos

#### 1.3.1 Extração do áudio e transcrição de áudio para texto

In [11]:
from audio_video_utils import extract_audio, extract_video_metadata, transcribe_audio

##### Execução

In [12]:
video_file_path = "../resources/Dica do professor.mp4"
audio_file_path = "../resources/Audio_Dica_do_Professor.wav"

In [13]:
# Extração dos metadados do vídeo
video_doc = extract_video_metadata(video_file_path)

# Extração do áudio em .wav
extract_audio(video_file_path, audio_file_path)

# Extração da transcrição do vídeo
transcript = transcribe_audio(audio_file_path)

video_doc['content'] = transcript
video_doc['keywords']=list(tfidf_keyword_extraction(transcript)),
video_doc['programming_languages']=extract_programming_language(transcript)
video_doc['video_path']=video_file_path
video_doc['format']='video'

MoviePy - Writing audio in ../resources/Audio_Dica_do_Professor.wav


MoviePy - Done.


In [14]:
video_doc

{'video_duration': '185.98',
 'video_size': '[426, 240]',
 'video_fps': '30',
 'video_start_time': '0',
 'video_end_time': '185.98',
 'content': 'Olá seja bem-vindo a dica do professor da unidade de aprendizagem criação de páginas web com HTML 5 Neste vídeo vamos falar sobre formatação de texto e âncoras em HTML a especificação HTML5 disponibiliza uma série de alimentos para formatar textos e criar âncoras em páginas web para isso você deve usar tags específicas que permitem formatar os elementos visando o equilíbrio do código com a semântica do HTML5 nesta dica criamos uma página HTML que apresenta uma relação de músicas de Roberto Carlos nesta página Montamos um menu de navegação que contém links para algumas páginas do Astro o objetivo desses links é posicionado a página no início de cada música sem precisar percorrer todo o documento vejamos agora a estrutura dessa página a instrução Doctor HTML e informa o navegador que estamos trabalhando com a versão 5 Roberto Carlos é o nosso r

### 1.4 Pré-processamento de imagens

In [15]:
import torch
from image_utils import get_image_description, get_exif_data, extract_text_from_image

In [16]:
image_path = '../resources/Infografico-1.jpg'

# Descrição do conteúdo visual
description = get_image_description(image_path)

# Extração de texto da imagem
custom_oem_psm_config = r'--oem 3 --psm 6'
image_text = extract_text_from_image(image_path, custom_oem_psm_config)

# Metadados EXIF caso encontre
exif_data = get_exif_data(image_path)


In [17]:
img_doc = {
    'content':image_text,
    'image_description':description,
    'image_path':image_path,
    'image_EXIF_data':exif_data,
    'format':'image'
}

In [18]:
img_doc

{'content': "| 000 https://vw'\nContém informações que são apresentadas Governo do Brasil\nno topo da página. Geralmente possui dados\nda organização e os links mais gerais do site,\ncomo página inicial, mapa do site, etc. PROGRAMA\nREDUÇÃO D\nPara ministros do (\nNAV: incentivo irá estim\nÉ uma área definida para os pa\nVo, a empreendedores.\nprincipais atalhos da página.\nAções vão bene\n50 mil crianças e\nA intenção é afasta\nContém informações que são apresentadas crime e diminuir a\nZ 2a e em toda a região\nno rodapé da página. Normalmente tem EiSdE VS\ndados de contato da organização ou alguma\noutra informação relevante. |\n* Acesso à informação\nO\n",
 'image_description': 'captura de ecrã de um ecrã de computador que apresenta uma lista de diferentes idiomas',
 'image_path': '../resources/Infografico-1.jpg',
 'image_EXIF_data': {},
 'format': 'image'}

### 1.5 Registros em mecanismo de busca/indexador whoosh

In [19]:
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID, STORED
from whoosh.analysis import StemmingAnalyzer
import os

In [20]:
schema = Schema(
    content=TEXT(stored=True, analyzer=StemmingAnalyzer()), 
    keywords=TEXT(stored=True), 
    relevant_phrases=TEXT(stored=True), 
    programming_languages=TEXT(stored=True), 
    format=ID(stored=True),
    text_path=ID(stored=True), 
    pdf_path=ID(stored=True), 
    video_path=ID(stored=True), 
    image_path=ID(stored=True),
    pdf_author=TEXT(stored=True),
    image_description=TEXT(stored=True),
    image_exif_data=STORED,
    video_duration=STORED,
    video_size=STORED,
    video_fps=STORED,
    video_start_time=STORED,
    video_end_time=STORED
    )

In [21]:
index_dir = "index"
if not os.path.exists(index_dir):
    os.mkdir(index_dir)
    ix = create_in(index_dir, schema)
else: 
    ix = open_dir(index_dir)


In [22]:
lock_file = os.path.join(index_dir, "MAIN_WRITELOCK")
if os.path.exists(lock_file):
    os.remove(lock_file)

In [23]:
documents = [text_doc, pdf_doc, video_doc, img_doc]

# O Whoosh não aceita listas como argumento, então uma pequenas conversão para string
def convert_list_to_string(value):
    if isinstance(value, tuple) and len(value) == 1 and isinstance(value[0], list):
        value = value[0]
    if isinstance(value, list):
        return ', '.join(value)
    return value

In [24]:
writer = ix.writer(timeout=2.0, delay=0.5)

for doc in documents:
    writer.add_document(
        content=convert_list_to_string(doc.get("content", "")),
        keywords=convert_list_to_string(doc.get("keywords", "")),
        relevant_phrases=convert_list_to_string(doc.get("relevant_phrases", "")),
        programming_languages=convert_list_to_string(doc.get("programming_languages", "")),
        format=convert_list_to_string(doc.get("format", "")),
        text_path=convert_list_to_string(doc.get("text_path", "")),
        pdf_path=convert_list_to_string(doc.get("pdf_path", "")),
        video_path=convert_list_to_string(doc.get("video_path", "")),
        image_path=convert_list_to_string(doc.get("image_path", "")),
        pdf_author=convert_list_to_string(doc.get("pdf_author", "")),
        image_description=convert_list_to_string(doc.get("image_description", "")),
        image_exif_data=convert_list_to_string(doc.get("image_exif_data", "")),
        video_duration=convert_list_to_string(doc.get("video_duration", "")),
        video_size=convert_list_to_string(doc.get("video_size", "")),
        video_fps=convert_list_to_string(doc.get("video_fps", "")),
        video_start_time=convert_list_to_string(doc.get("video_start_time", "")),
        video_end_time=convert_list_to_string(doc.get("video_end_time", "")),
    )

writer.commit()

In [25]:
with ix.searcher() as searcher:
    for docnum in range(searcher.reader().doc_count_all()):
        doc = searcher.reader().stored_fields(docnum)
        print("Documento {}: {}".format(docnum, doc))

Documento 0: {'content': 'Sistemas desenvolvidos para a plataforma web utilizam a linguagem HTML para a exibição de conteúdo, tanto para páginas estáticas como dinâmicas. Com HTML5 é possível criar páginas web com diversos recursos para a apresentação de dados por meio de novas marcações que permitem o uso de semântica e acessibilidade, facilitando a pesquisa por motores de busca automática e dispositivos próprios para deficientes visuais e auditivos.\n\nNesta Unidade de Aprendizagem, você vai estudar a estrutura de páginas web, a formatação de texto em documentos hipertexto e a apresentação de links, listas e tabelas em HTML5.\n\nBons estudos.\n\nAo final desta Unidade de Aprendizagem, você deve apresentar os seguintes aprendizados:\nDefinir a estrutura de uma página web com HTML5.\nAplicar formatação de texto em uma página web com HTML5.\nDesenvolver listas e tabelas em uma página web com HTML5.', 'format': 'text', 'image_description': '', 'image_exif_data': '', 'image_path': '', 'ke

## Etapa 2: Prompt de aprendizagem adaptativa

Enunciado: Construa um prompt interativo, utilizando as tecnologias que julgar apropriadas, que identifique as dificuldades e lacunas de conhecimento dos usuários em um diálogo fluido e intuitivo para avaliar e entender as áreas onde seu conhecimento sobre um tema específico pode ser insuficiente. O escopo deve estar limitado ao conteúdo indexado. Durante as interações, inclua mecanismos que permitam identificar as preferências dos usuários quanto ao formato de aprendizado mais efetivo para eles, seja texto, vídeo ou áudio, adaptando-se assim às suas preferências pessoais de consumo de conteúdo.

Baseado nas interações analise as dificuldades e gere conteúdos dinâmicos curtos em diferentes formatos (vídeos, áudios, textos) para abordar as necessidades específicas de aprendizagem do usuário. Os conteúdos devem ser relevantes, informativos e adaptados ao nível de conhecimento do usuário.

**Nota: Idealmente, eu usaria uma vector store para cálculo da semelhança semântica do questionamento do aluno com o conteúdo encontrado após filtrar por formatos, mas não foi possível implementar localmente**

In [36]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from whoosh.qparser import MultifieldParser, OrGroup
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-large-portuguese-cased")
model = AutoModelForCausalLM.from_pretrained("neuralmind/bert-large-portuguese-cased")

In [38]:
question = "Como utilizar HTML5 para criar uma página web?"
format_preference = "video"

In [39]:
# Função para gerar texto usando o modelo LLM ao final da esteira
def generate_text(prompt, num_tokens=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=num_tokens, num_return_sequences=1, temperature=0.7)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Função para buscar no índice
def search_index(query, format_preference):
    ix = open_dir(index_dir)
    with ix.searcher() as searcher:
        query_parser = MultifieldParser(["content", "keywords"], ix.schema, group=OrGroup)
        myquery = query_parser.parse(query)
        results = searcher.search(myquery)
        filtered_results = [hit for hit in results if hit['format'] == format_preference]
        return filtered_results[0]['content'] if filtered_results else "Nenhum conteúdo relevante encontrado"

content = search_index(question, format_preference)
print(content)


Consulta parseada: (content:como OR keywords:como OR content:utilizar OR keywords:utilizar OR content:html5 OR keywords:html5 OR content:para OR keywords:para OR content:criar OR keywords:criar OR content:uma OR keywords:uma OR content:página OR keywords:página OR content:web? OR keywords:web?)
Resultados: <Top 4 Results for Or([Term('content', 'como'), Term('keywords', 'como'), Term('content', 'utilizar'), Term('keywords', 'utilizar'), Term('content', 'html5'), Term('keywords', 'html5'), Term('content', 'para'), Term('keywords', 'para'), Term('content', 'criar'), Term('keywords', 'criar'), Term('content', 'uma'), Term('keywords', 'uma'), Term('content', 'página'), Term('keywords', 'página'), Wildcard('content', 'web?'), Wildcard('keywords', 'web?')]) runtime=0.003389900000001944>
Olá seja bem-vindo a dica do professor da unidade de aprendizagem criação de páginas web com HTML 5 Neste vídeo vamos falar sobre formatação de texto e âncoras em HTML a especificação HTML5 disponibiliza uma 

In [40]:
if content == "Nenhum conteúdo relevante encontrado":
    generated_text = generate_text(question)
    print(generated_text)
else:
    print("Conteúdo recomendado:", content)

Conteúdo recomendado: Olá seja bem-vindo a dica do professor da unidade de aprendizagem criação de páginas web com HTML 5 Neste vídeo vamos falar sobre formatação de texto e âncoras em HTML a especificação HTML5 disponibiliza uma série de alimentos para formatar textos e criar âncoras em páginas web para isso você deve usar tags específicas que permitem formatar os elementos visando o equilíbrio do código com a semântica do HTML5 nesta dica criamos uma página HTML que apresenta uma relação de músicas de Roberto Carlos nesta página Montamos um menu de navegação que contém links para algumas páginas do Astro o objetivo desses links é posicionado a página no início de cada música sem precisar percorrer todo o documento vejamos agora a estrutura dessa página a instrução Doctor HTML e informa o navegador que estamos trabalhando com a versão 5 Roberto Carlos é o nosso rei é o título da nossa página ele é exibido na barra de títulos do navegador músicas de Roberto Carlos é o título principal 

## Outros passos/soluções desejadas

- Montar realmente uma estrutura de prompt em que a IA trataria e reconheceria a linguagem de busca também com a dificuldade (Como pode se ver acima, a solução ficou em um mecanismo de busca na parte da linguagem), realizar busca semântica criando vector store, incorporando um personagem de professor à IA por prompt engineering